# Convolutional Neural Network in Implemetation

### Importing Libraries

In [4]:
import torch
from torch import nn
from torch.utils.data import TensorDataset, Dataset, DataLoader
from torch.optim import SGD, Adam
from torchvision import datasets
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [8]:
X_train = torch.tensor([[[[1,2,3,4],[2,3,4,5], \
                          [5,6,7,8],[1,3,4,5]]], \
                        [[[-1,2,3,-4],[2,-3,4,5], \
                          [-5,6,-7,8],[-1,-3,-4,-5]]]]).to(device).float()
X_train /= 8
y_train = torch.tensor([0,1]).to(device).float()

torch.Size([2, 1, 4, 4])

### Model 

In [5]:
def get_model():
  model = nn.Sequential(
      nn.Conv2d(1, 1, kernel_size=3),
      nn.MaxPool2d(2),
      nn.ReLU(),
      nn.Flatten(),
      nn.Linear(1, 1),
      nn.Sigmoid(),
  ).to(device)

  loss_fn = nn.BCELoss()
  optimizer = Adam(model.parameters(), lr=1e-3)
  return model, loss_fn, optimizer

In [11]:
from torchsummary import summary

model, loss_fn, optimizer = get_model()
summary(model, X_train[0].shape)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1              [-1, 1, 2, 2]              10
         MaxPool2d-2              [-1, 1, 1, 1]               0
              ReLU-3              [-1, 1, 1, 1]               0
           Flatten-4                    [-1, 1]               0
            Linear-5                    [-1, 1]               2
           Sigmoid-6                    [-1, 1]               0
Total params: 12
Trainable params: 12
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


### training

In [12]:
def train_batch(x, y, model, opt, loss_fn):
  model.train()
  prediction = model(x)

  batch_loss = loss_fn(prediction.squeeze(0), y)
  batch_loss.backward()

  optimizer.step()
  optimizer.zero_grad()

  return batch_loss.item()

In [13]:
train_dl = DataLoader(TensorDataset(X_train, y_train)) # no need of separate class

In [16]:
for epoch in range(2000):
  for ix, batch in enumerate(iter(train_dl)):
    x, y = batch
    batch_loss = train_batch(x, y, model, optimizer, loss_fn)


In [18]:
list(model.children())

[Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1)),
 MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
 ReLU(),
 Flatten(start_dim=1, end_dim=-1),
 Linear(in_features=1, out_features=1, bias=True),
 Sigmoid()]